In [1]:
import psycopg2
import random
import time
from time import strftime, localtime

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.impute import SimpleImputer

print("Start: %s" % strftime("%b %d %Y %H:%M:%S", localtime()))
try:
    conn = psycopg2.connect("dbname='homecredit' user='postgres' host='localhost' password='metis'")
except w:
    print(w)
    print ("I am unable to connect to the database")

Start: Oct 25 2018 22:40:24


In [2]:
%time application_train = pd.read_csv("data/application_train.csv")


CPU times: user 4.26 s, sys: 540 ms, total: 4.8 s
Wall time: 4.55 s


In [3]:
application_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
cols = list(application_train.columns)
cols

['SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_A

In [5]:
binary = []
probly_categorical = []
probly_numeric = []

for col in cols:
    uniques = len(application_train[col].unique())
    is_numeric = np.issubdtype(application_train[col].dtype, np.number)

    if uniques == 2: # binary whether numeric or not
        binary.append(col)
    elif not is_numeric: # categorical
        probly_categorical.append(col)
    elif uniques < 10: # numeric but only a few uniques
        probly_categorical.append(col)
    else:
        probly_numeric.append(col)
        print("%30s: %d" % (col, uniques))



                    SK_ID_CURR: 307511
                  CNT_CHILDREN: 15
              AMT_INCOME_TOTAL: 2548
                    AMT_CREDIT: 5603
                   AMT_ANNUITY: 13673
               AMT_GOODS_PRICE: 1003
    REGION_POPULATION_RELATIVE: 81
                    DAYS_BIRTH: 17460
                 DAYS_EMPLOYED: 12574
             DAYS_REGISTRATION: 15688
               DAYS_ID_PUBLISH: 6168
                   OWN_CAR_AGE: 63
               CNT_FAM_MEMBERS: 18
       HOUR_APPR_PROCESS_START: 24
                  EXT_SOURCE_1: 114585
                  EXT_SOURCE_2: 119832
                  EXT_SOURCE_3: 815
                APARTMENTS_AVG: 2340
              BASEMENTAREA_AVG: 3781
   YEARS_BEGINEXPLUATATION_AVG: 286
               YEARS_BUILD_AVG: 150
                COMMONAREA_AVG: 3182
                 ELEVATORS_AVG: 258
                 ENTRANCES_AVG: 286
                 FLOORSMAX_AVG: 404
                 FLOORSMIN_AVG: 306
                  LANDAREA_AVG: 3528
        

In [6]:
binary

['TARGET',
 'NAME_CONTRACT_TYPE',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21']

In [7]:
probly_categorical

['CODE_GENDER',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE',
 'AMT_REQ_CREDIT_BUREAU_HOUR']

In [8]:
categorical = ['CODE_GENDER',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE',
 'OCCUPATION_TYPE',
 'ORGANIZATION_TYPE',
]             


In [9]:
application_train.groupby(['AMT_REQ_CREDIT_BUREAU_HOUR']).count()


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
AMT_REQ_CREDIT_BUREAU_HOUR,,,,,,,,,,,,,,,,,,,,,
0.0,264366,264366,264366,264366,264366,264366,264366,264366,264366,264355,...,264366,264366,264366,264366,264366,264366,264366,264366,264366,264366
1.0,1560,1560,1560,1560,1560,1560,1560,1560,1560,1560,...,1560,1560,1560,1560,1560,1560,1560,1560,1560,1560
2.0,56,56,56,56,56,56,56,56,56,56,...,56,56,56,56,56,56,56,56,56,56
3.0,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
4.0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [10]:
binary = ['TARGET',
 'NAME_CONTRACT_TYPE',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21']

In [11]:
probly_numeric

['SK_ID_CURR',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'CNT_FAM_MEMBERS',
 'HOUR_APPR_PROCESS_START',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI'

In [12]:
numerical = ['SK_ID_CURR',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'CNT_FAM_MEMBERS',
 'HOUR_APPR_PROCESS_START',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD_MEDI',
 'COMMONAREA_MEDI',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'FLOORSMAX_MEDI',
 'FLOORSMIN_MEDI',
 'LANDAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAREA_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAREA_MEDI',
 'TOTALAREA_MODE',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'DAYS_LAST_PHONE_CHANGE',
 'AMT_REQ_CREDIT_BUREAU_HOUR',
 'AMT_REQ_CREDIT_BUREAU_DAY',
 'AMT_REQ_CREDIT_BUREAU_WEEK',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'AMT_REQ_CREDIT_BUREAU_QRT',
 'AMT_REQ_CREDIT_BUREAU_YEAR']



In [13]:
nrows, ncols = application_train.shape
nrows, ncols

(307511, 122)

In [14]:
# sample 10,000 row indexesfrom application_train (or use pd.sample)
#subsample_indexes = sorted(random.sample(range(nrows), 10000))
subsample_indexes=range(nrows)
subsample = application_train.loc[subsample_indexes, :]
subsample.shape

(307511, 122)

In [15]:
#categorical vars
subsample.loc[:,categorical]

,CODE_GENDER,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,FONDKAPREMONT_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OCCUPATION_TYPE,ORGANIZATION_TYPE
0,M,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,2,2,WEDNESDAY,reg oper account,block of flats,"Stone, brick",No,Laborers,Business Entity Type 3
1,F,Family,State servant,Higher education,Married,House / apartment,1,1,MONDAY,reg oper account,block of flats,Block,No,Core staff,School
2,M,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,2,2,MONDAY,NaN,NaN,NaN,NaN,Laborers,Government
3,F,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,2,2,WEDNESDAY,NaN,NaN,NaN,NaN,Laborers,Business Entity Type 3
4,M,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,2,2,THURSDAY,NaN,NaN,NaN,NaN,Core staff,Religion
5,M,"Spouse, partner",State servant,Secondary / secondary special,Married,House / apartment,2,2,WEDNESDAY,NaN,NaN,NaN,NaN,Laborers,Other
6,F,Unaccompanied,Commercial associate,Higher education,Married,House / apartment,2,2,SUNDAY,NaN,NaN,NaN,NaN,Accountants,Business Entity Type 3
7,M,Unaccompanied,State servant,Higher education,Married,House / apartment,3,3,MONDAY,NaN,NaN,NaN,NaN,Managers,Other
8,F,Children,Pensioner,Secondary / secondary special,Married,House / apartment,2,2,WEDNESDAY,NaN,NaN,NaN,NaN,NaN,XNA
9,M,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,2,2,THURSDAY,NaN,NaN,NaN,NaN,Laborers,Electricity


In [16]:
# make dummies
dummies = pd.get_dummies(subsample.loc[:,categorical])
dummies.head()


,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,...,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA
0,2,2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,2,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:

subsample.loc[:,binary]
subsample.groupby("NAME_CONTRACT_TYPE").count()


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
NAME_CONTRACT_TYPE,,,,,,,,,,,,,,,,,,,,,
Cash loans,278232,278232,278232,278232,278232,278232,278232,278232,278220,278232,...,278232,278232,278232,278232,240993,240993,240993,240993,240993,240993
Revolving loans,29279,29279,29279,29279,29279,29279,29279,29279,29279,29001,...,29279,29279,29279,29279,24999,24999,24999,24999,24999,24999


In [18]:
subsample["NAME_CONTRACT_TYPE"] = (subsample["NAME_CONTRACT_TYPE"] == "Revolving loans")
subsample["FLAG_OWN_CAR"] = (subsample["FLAG_OWN_CAR"] == "Y")
subsample["FLAG_OWN_REALTY"] = (subsample["FLAG_OWN_REALTY"] == "Y")
subsample.groupby("NAME_CONTRACT_TYPE").count()


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
NAME_CONTRACT_TYPE,,,,,,,,,,,,,,,,,,,,,
False,278232,278232,278232,278232,278232,278232,278232,278232,278220,278232,...,278232,278232,278232,278232,240993,240993,240993,240993,240993,240993
True,29279,29279,29279,29279,29279,29279,29279,29279,29279,29001,...,29279,29279,29279,29279,24999,24999,24999,24999,24999,24999


In [19]:
# drop categorical columns from subsample, add dummies
len(subsample.columns), len(categorical), len(dummies.columns)


(122, 15, 136)

In [20]:
122-14+56

164

In [21]:
for col in categorical:
    print(col)
    subsample.drop(col,axis=1, inplace=True)

subsample.shape


CODE_GENDER
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
REGION_RATING_CLIENT
REGION_RATING_CLIENT_W_CITY
WEEKDAY_APPR_PROCESS_START
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE
OCCUPATION_TYPE
ORGANIZATION_TYPE


(307511, 107)

In [22]:
binary_df = subsample.loc[:,binary]

# binarize binary values
for col in binary:
    binary_df[col] = LabelBinarizer().fit_transform(binary_df[[col]])

binary_df.drop("TARGET",axis=1, inplace=True)
binary_df.tail()


,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,REG_REGION_NOT_LIVE_REGION,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
307506,0,0,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307507,0,0,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
307508,0,0,1,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
307509,0,0,1,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
307510,0,0,0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
numerical_df = subsample.loc[:,numerical]

# impute missing values
for col in numerical:
    numerical_df[col] = SimpleImputer(missing_values=np.nan, strategy='mean').fit_transform(numerical_df[[col]])

numerical_df.drop("SK_ID_CURR",axis=1, inplace=True)
numerical_df.tail()
    


,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,...,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
307506,0.0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327.0,-236.0,-8456.0,-1982.0,...,0.0,0.0,0.0,-273.0,0.006402,0.007,0.034362,0.267395,0.265474,1.899974
307507,0.0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775.0,365243.0,-4388.0,-4090.0,...,0.0,0.0,0.0,0.0,0.006402,0.007,0.034362,0.267395,0.265474,1.899974
307508,0.0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966.0,-7921.0,-6737.0,-5150.0,...,0.0,6.0,0.0,-1909.0,1.000000,0.000,0.000000,1.000000,0.000000,1.000000
307509,0.0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961.0,-4786.0,-2562.0,-931.0,...,0.0,0.0,0.0,-322.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000
307510,0.0,157500.0,675000.0,49117.5,675000.0,0.046220,-16856.0,-1262.0,-5128.0,-410.0,...,0.0,0.0,0.0,-787.0,0.000000,0.000,0.000000,2.000000,0.000000,1.000000


In [24]:
# put all categorical and binary at the end for lightgbm

subsample = pd.concat((subsample[["TARGET"]], numerical_df, dummies, binary_df), axis=1)
subsample.shape

(307511, 242)

In [25]:
subsample.to_pickle("subsample.pickle")

In [26]:
subsample.head()


,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21
0,1,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,-3648.0,...,0,0,0,0,0,0,0,0,0,0
1,0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,-1186.0,...,0,0,0,0,0,0,0,0,0,0
2,0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,-4260.0,...,0,0,0,0,0,0,0,0,0,0
3,0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,-9833.0,...,0,0,0,0,0,0,0,0,0,0
4,0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,-4311.0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
subsample.count()

TARGET                         307511
CNT_CHILDREN                   307511
AMT_INCOME_TOTAL               307511
AMT_CREDIT                     307511
AMT_ANNUITY                    307511
AMT_GOODS_PRICE                307511
REGION_POPULATION_RELATIVE     307511
DAYS_BIRTH                     307511
DAYS_EMPLOYED                  307511
DAYS_REGISTRATION              307511
DAYS_ID_PUBLISH                307511
OWN_CAR_AGE                    307511
CNT_FAM_MEMBERS                307511
HOUR_APPR_PROCESS_START        307511
EXT_SOURCE_1                   307511
EXT_SOURCE_2                   307511
EXT_SOURCE_3                   307511
APARTMENTS_AVG                 307511
BASEMENTAREA_AVG               307511
YEARS_BEGINEXPLUATATION_AVG    307511
YEARS_BUILD_AVG                307511
COMMONAREA_AVG                 307511
ELEVATORS_AVG                  307511
ENTRANCES_AVG                  307511
FLOORSMAX_AVG                  307511
FLOORSMIN_AVG                  307511
LANDAREA_AVG

In [29]:
subsample["AMT_REQ_CREDIT_BUREAU_HOUR"]

0         0.000000
1         0.000000
2         0.000000
3         0.006402
4         0.000000
5         0.000000
6         0.000000
7         0.000000
8         0.000000
9         0.006402
10        0.000000
11        0.000000
12        0.000000
13        0.000000
14        0.006402
15        0.000000
16        0.000000
17        0.006402
18        0.000000
19        0.000000
20        0.006402
21        0.000000
22        0.000000
23        0.000000
24        0.000000
25        0.000000
26        0.000000
27        0.000000
28        0.000000
29        0.006402
            ...   
307481    0.000000
307482    0.000000
307483    0.000000
307484    0.006402
307485    0.000000
307486    0.000000
307487    0.000000
307488    0.000000
307489    0.000000
307490    0.000000
307491    0.000000
307492    0.000000
307493    0.000000
307494    0.000000
307495    0.000000
307496    0.000000
307497    0.000000
307498    0.000000
307499    0.000000
307500    0.000000
307501    0.006402
307502    0.